In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

In [2]:
# Combining dataset for all 12 months into multiple dataframes for addresss, bnf and gp data
month_list = ['jan','feb','mar','apr','may','june','july','aug','sep','oct','nov','dec']
num_list = ['01','02','03','04','05','06','07','08','09','10','11','12']

combined_addr = pd.DataFrame()
combined_bnf = pd.DataFrame()
combined_gp = pd.DataFrame()
for num, month in zip(num_list,month_list):
    data_addr = pd.read_csv(f'GP_2020/gp_data_{month}_2020/Address.csv')
    data_bnf = pd.read_csv(f'GP_2020/gp_data_{month}_2020/BNF.csv')
    data_gp = pd.read_csv(f'GP_2020/gp_data_{month}_2020/GPData2020{num}.csv')
    combined_addr = pd.concat([combined_addr,data_addr])
    combined_bnf = pd.concat([combined_bnf,data_bnf])
    combined_gp = pd.concat([combined_gp,data_gp])

In [3]:
combined_addr = combined_addr.drop_duplicates().reset_index(drop=True)
combined_bnf = combined_bnf.drop_duplicates().reset_index(drop=True)
combined_gp = combined_gp.drop_duplicates().reset_index(drop=True)

In [4]:
combined_bnf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5698 entries, 0 to 5697
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   BNFChapter       5698 non-null   int64 
 1    ChapterDesc     5698 non-null   object
 2    BNFSection      5698 non-null   int64 
 3    SectionDesc     5698 non-null   object
 4    BNFSubSection   5698 non-null   int64 
 5    SubSectionDesc  5698 non-null   object
 6    BNFChemical     5698 non-null   object
 7    ChemicalDesc    5698 non-null   object
dtypes: int64(3), object(5)
memory usage: 356.2+ KB


In [5]:
# Viewing count of chapter descriptions before data cleaning
combined_bnf[' ChapterDesc'].value_counts()

Central Nervous System                      582
Stoma Appliances                            568
Cardiovascular System                       504
Skin                                        463
Infections                                  460
Nutrition And Blood                         322
Endocrine System                            309
Gastro-Intestinal System                    282
Appliances                                  254
Respiratory System                          246
Incontinence Appliances                     241
Malignant Disease & Immunosuppression       232
Eye                                         215
Dressings                                   173
Ear  Nose And Oropharynx                    166
Musculoskeletal & Joint Diseases            164
Obstetrics Gynae+Urinary Tract Disorders    151
Anaesthesia                                  71
Other Drugs And Preparations                 63
Immunological Products & Vaccines            59
Nutrition and Blood                     

In [6]:
# Data cleaning on column names and chapter description column
combined_bnf.columns = combined_bnf.columns.map(lambda x: x.strip())
sub_combined_bnf = combined_bnf.iloc[:,[1,6]].drop_duplicates().reset_index(drop=True)
sub_combined_bnf['ChapterDesc'] = sub_combined_bnf['ChapterDesc'].apply(lambda x: x.strip().replace('&','and').replace('And','and'))
sub_combined_bnf['ChapterDesc'] = sub_combined_bnf['ChapterDesc'].apply(lambda x: 'Obstetrics, Gynaecology and Urinary-Tract Disorders' if x.find('Obstetrics')!=-1 else x)
sub_combined_bnf['ChapterDesc'] = sub_combined_bnf['ChapterDesc'].apply(lambda x: 'Ear, Nose and Oropharynx' if x.find('Ear')!=-1 else x)
sub_combined_bnf = sub_combined_bnf.drop_duplicates().reset_index(drop=True)
sub_combined_bnf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4717 entries, 0 to 4716
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ChapterDesc  4717 non-null   object
 1   BNFChemical  4717 non-null   object
dtypes: object(2)
memory usage: 73.8+ KB


In [7]:
# Viewing count of chapter descriptions after data cleaning
sub_combined_bnf['ChapterDesc'].value_counts()

Stoma Appliances                                       566
Central Nervous System                                 406
Skin                                                   387
Infections                                             381
Cardiovascular System                                  371
Nutrition and Blood                                    296
Endocrine System                                       248
Incontinence Appliances                                238
Appliances                                             230
Malignant Disease and Immunosuppression                223
Gastro-Intestinal System                               222
Respiratory System                                     197
Eye                                                    196
Ear, Nose and Oropharynx                               161
Musculoskeletal and Joint Diseases                     149
Dressings                                              139
Obstetrics, Gynaecology and Urinary-Tract Disorders    1

In [8]:
combined_addr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9357 entries, 0 to 9356
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Period      9357 non-null   int64 
 1   PracticeId  9345 non-null   object
 2   Locality    372 non-null    object
 3   Street      9357 non-null   object
 4   Area        9357 non-null   object
 5   Posttown    9309 non-null   object
 6   County      8670 non-null   object
 7   Postcode    9357 non-null   object
dtypes: int64(1), object(7)
memory usage: 584.9+ KB


In [9]:
# Create full address column
combined_addr[['Street','Area','Posttown','County']] = combined_addr[['Street','Area','Posttown','County']].applymap(lambda x: x.strip().title() if type(x)!=float else x)
combined_addr['Full address'] = combined_addr['Street'] + ', ' + combined_addr['Area'] + ', ' + combined_addr['Posttown'].apply(lambda x: x + ', ' if type(x)!=float and x!='' else '') + combined_addr['County'].apply(lambda x: x + ', ' if type(x)!=float and x!='' else '') + combined_addr['Postcode'] 

In [10]:
# Data cleaning on practice id and postcode columns
combined_addr['PracticeId'] = combined_addr['PracticeId'].apply(lambda x: x.strip() if type(x)!=float else x)
combined_addr['Postcode'] = combined_addr['Postcode'].apply(lambda x: x.replace(" ","").strip() if type(x)!=float else x)
sub_combined_addr = combined_addr.drop(['Street','Area','Posttown','Locality','County'],axis=1).drop_duplicates()
sub_combined_addr

,Period,PracticeId,Postcode,Full address
0,202001,W93001,NP115GX,"St. Luke'S Surgery, (Off Gwyddon Road), Aberca..."
1,202001,W93004,NP75DL,"Tudor Gate Surgery, Tudor Street, Abergavenny,..."
2,202001,W93008,NP132AB,"Aberbeeg Medical Centre, The Square, Aberbeeg ..."
3,202001,W94635,LL748TF,"Gerafon Surgery, Benllech, Ynys Mon, LL74 8TF"
4,202001,W93017,NP234EY,"Essendene Surgery, Worcester Street, Brynmawr,..."
...,...,...,...,...
9352,202012,6B9,NP183XQ,"Wentwood Ward, St Cadocs Hospital, Lodge Road,..."
9353,202012,6C1,LL228DP,"Ivanhoe Building, Abergele Hospital, Llanfair ..."
9354,202012,6C2,NP131DB,"Anvil Court, Church Street, Abertillery, Gwent..."
9355,202012,6C3,NP40YP,"Mamhilad House, Mamhilad Park Est., Pontypool,..."


In [11]:
sub_combined_addr['PracticeId'].value_counts()

W95041    13
W93001    12
W97056    12
W98058    12
W95012    12
          ..
W00179     3
W00182     2
W00183     2
W00181     2
W00180     1
Name: PracticeId, Length: 784, dtype: int64

In [12]:
# Both postcodes provide same county
sub_combined_addr[sub_combined_addr['PracticeId'] == 'W95041']

,Period,PracticeId,Postcode,Full address
201,202001,W95041,CF363XB,"The Portway Surgery, 1 The Portway, Porthcawl ..."
975,202002,W95041,CF363XB,"The Portway Surgery, 1 The Portway, Porthcawl ..."
1749,202003,W95041,CF363XB,"The Portway Surgery, 1 The Portway, Porthcawl ..."
2528,202004,W95041,CF363XB,"The Portway Surgery, 1 The Portway, Porthcawl ..."
3307,202005,W95041,CF363XB,"The Portway Surgery, 1 The Portway, Porthcawl ..."
4086,202006,W95041,CF363XB,"The Portway Surgery, 1 The Portway, Porthcawl ..."
4865,202007,W95041,CF363XB,"The Portway Surgery, 1 The Portway, Porthcawl ..."
5643,202008,W95041,CF363XB,"The Portway Surgery, 1 The Portway, Porthcawl ..."
6421,202009,W95041,CF363XB,"The Portway Surgery, 1 The Portway, Porthcawl ..."
7202,202010,W95041,CF363XB,"The Portway Surgery, 1 The Portway, Porthcawl ..."


In [13]:
# Removing additional row since both postcodes provide same county
sub_combined_addr = sub_combined_addr.drop(9312)
sub_combined_addr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9344 entries, 0 to 9356
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Period        9344 non-null   int64 
 1   PracticeId    9332 non-null   object
 2   Postcode      9344 non-null   object
 3   Full address  9344 non-null   object
dtypes: int64(1), object(3)
memory usage: 365.0+ KB


In [14]:
combined_gp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7161786 entries, 0 to 7161785
Data columns (total 12 columns):
 #   Column      Dtype  
---  ------      -----  
 0   HB          object 
 1   Locality    object 
 2   PracticeID  object 
 3   BNFCode     object 
 4   BNFName     object 
 5   Items       int64  
 6   NIC         float64
 7   ActCost     float64
 8   Quantity    int64  
 9   DDD         float64
 10  ADQ         float64
 11  Period      int64  
dtypes: float64(4), int64(3), object(5)
memory usage: 655.7+ MB


In [15]:
combined_gp.isnull().sum()

HB            0
Locality      0
PracticeID    0
BNFCode       0
BNFName       0
Items         0
NIC           0
ActCost       0
Quantity      0
DDD           0
ADQ           0
Period        0
dtype: int64

In [16]:
# Extract first 9 characters from BNF Code
combined_gp['BNFCode_9'] = combined_gp['BNFCode'].apply(lambda x: x[:9])

In [17]:
combined_data = sub_combined_bnf.merge(combined_gp,left_on='BNFChemical',
                                       right_on='BNFCode_9',how='inner') \
                .merge(sub_combined_addr, left_on=['Period','PracticeID'],
                       right_on=['Period','PracticeId'],how='left') \
                .rename({'Postcode':'GP_Postcode','PracticeId':'GP_PracticeID',
                         'Full address': 'GP_Address'},axis=1) \
                .merge(sub_combined_addr, left_on=['Period','Locality'], 
                       right_on=['Period','PracticeId'],how='left') \
                .rename({'Postcode':'LO_Postcode','PracticeId':'LO_PracticeID', 
                         'Full address': 'LO_Address'},axis=1) \
                .merge(sub_combined_addr, left_on=['Period','HB'],
                       right_on=['Period','PracticeId'],how='left') \
                .rename({'Postcode':'HB_Postcode','PracticeId':'HB_PracticeID', 
                         'Full address': 'HB_Address'},axis=1)

In [18]:
# Web scraping county data for HB Ids
actual_HB_county = []
i=0
start = time.time()
for code in combined_data.HB_Postcode.unique():
    r = requests.get(f"https://checkmypostcode.uk/{code}")
    soup = BeautifulSoup(r.content)
    result = []
    for a_tag in soup.find_all('div', class_='medium-4 columns')[0]:
        result.append(a_tag)
    try:
        county = str(result[3]).replace("<p>","").replace("</p>","").replace("Council","").strip()
    except:
        county = None
        print(f'{code} has issues')
    actual_HB_county.append(county)
    i+=1
    print(f'{i} pages completed in {time.time()-start} seconds')

1 pages completed in 2.7155568599700928 seconds
2 pages completed in 5.276043176651001 seconds
3 pages completed in 7.034479379653931 seconds
CF144TT has issues
4 pages completed in 11.888298511505127 seconds
SA313YH has issues
5 pages completed in 13.38974404335022 seconds
6 pages completed in 17.441293716430664 seconds
7 pages completed in 19.771069288253784 seconds


In [19]:
# Web scraping county data for Locality IDs
actual_LO_county = []
i=0
start = time.time()
for code in combined_data.LO_Postcode.unique():
    r = requests.get(f"https://checkmypostcode.uk/{code}")
    soup = BeautifulSoup(r.content)
    result = []
    for a_tag in soup.find_all('div', class_='medium-4 columns')[0]:
        result.append(a_tag)
    try:
        county = str(result[3]).replace("<p>","").replace("</p>","").replace("Council","").strip()
    except:
        county = None
        print(f'{code} has issues')
    actual_LO_county.append(county)
    i+=1
    print(f'{i} pages completed in {time.time()-start} seconds')

1 pages completed in 7.187670707702637 seconds
2 pages completed in 10.46579909324646 seconds
3 pages completed in 22.24811816215515 seconds
4 pages completed in 24.430455446243286 seconds
5 pages completed in 25.89770531654358 seconds
6 pages completed in 28.548630237579346 seconds
7 pages completed in 30.37105965614319 seconds
8 pages completed in 37.53422474861145 seconds
9 pages completed in 38.08319592475891 seconds
CF144TT has issues
10 pages completed in 38.61469745635986 seconds
11 pages completed in 41.6185884475708 seconds
SA487HA has issues
12 pages completed in 42.8581862449646 seconds
13 pages completed in 45.18166899681091 seconds
14 pages completed in 48.03002095222473 seconds
15 pages completed in 52.632062911987305 seconds
16 pages completed in 62.69592809677124 seconds
17 pages completed in 67.83401703834534 seconds
18 pages completed in 68.97210216522217 seconds


In [20]:
# Web scraping county data for General Practitioner IDs
actual_GP_county = []
i=0
start = time.time()
for code in combined_data[combined_data['GP_Postcode'].notnull()]['GP_Postcode'].unique():
    r = requests.get(f"https://checkmypostcode.uk/{code}")
    soup = BeautifulSoup(r.content)
    result = []
    for a_tag in soup.find_all('div', class_='medium-4 columns')[0]:
        result.append(a_tag)
    try:
        county = str(result[3]).replace("<p>","").replace("</p>","").replace("Council","").strip()
    except:
        county = None
        print(f'{code} has issues')
    actual_GP_county.append(county)
    i+=1
    if i%10 == 0:
        print(f'{i} pages completed in {time.time()-start} seconds')

10 pages completed in 49.29631304740906 seconds
20 pages completed in 97.5466239452362 seconds
30 pages completed in 131.1309781074524 seconds
40 pages completed in 162.83986639976501 seconds
50 pages completed in 237.05996417999268 seconds
60 pages completed in 282.42004919052124 seconds
70 pages completed in 328.4894108772278 seconds
80 pages completed in 360.4391071796417 seconds
90 pages completed in 406.19662070274353 seconds
100 pages completed in 440.5357503890991 seconds
110 pages completed in 482.58088755607605 seconds
120 pages completed in 516.1402418613434 seconds
130 pages completed in 550.529465675354 seconds
140 pages completed in 576.1472985744476 seconds
150 pages completed in 614.2226002216339 seconds
160 pages completed in 666.5015993118286 seconds
170 pages completed in 703.9978289604187 seconds
180 pages completed in 741.4878327846527 seconds
190 pages completed in 782.7038097381592 seconds
200 pages completed in 809.3466093540192 seconds
NP447XX has issues
210 pag

In [21]:
gpcountydata = pd.DataFrame({'GP_Postcode':pd.Series(combined_data[combined_data['GP_Postcode'].notnull()]['GP_Postcode'].unique()),
             'GP_County':pd.Series(actual_GP_county)})
locountydata = pd.DataFrame({'LO_Postcode':combined_data.LO_Postcode.unique(),'LO_County':pd.Series(actual_LO_county)})
hbcountydata = pd.DataFrame({'HB_Postcode':combined_data.HB_Postcode.unique(),'HB_County':pd.Series(actual_HB_county)})

In [22]:
gpcountydata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 481 entries, 0 to 480
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   GP_Postcode  481 non-null    object
 1   GP_County    473 non-null    object
dtypes: object(2)
memory usage: 7.6+ KB


In [23]:
locountydata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   LO_Postcode  18 non-null     object
 1   LO_County    16 non-null     object
dtypes: object(2)
memory usage: 416.0+ bytes


In [24]:
hbcountydata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   HB_Postcode  7 non-null      object
 1   HB_County    5 non-null      object
dtypes: object(2)
memory usage: 240.0+ bytes


In [25]:
# Defunct postcodes
gpcountydata[gpcountydata['GP_County'].isnull()]

,GP_Postcode,GP_County
202,NP447XX,None
278,NP223XP,None
340,LD3OAW,None
390,LL163TH,None
391,LL163BL,None
397,SA487HA,None
458,OO000OO,None
477,SA15BE,None


In [26]:
# Checking for county of postcode LD3OAW
combined_addr[combined_addr['Postcode'] == 'LD3OAW']

,Period,PracticeId,Locality,Street,Area,Posttown,County,Postcode,Full address
493,202001,W96017,NaN,Haygarth Medical Centre,Hay Road,Talgarth Brecon,Powys,LD3OAW,"Haygarth Medical Centre, Hay Road, Talgarth Br..."
1267,202002,W96017,NaN,Haygarth Medical Centre,Hay Road,Talgarth Brecon,Powys,LD3OAW,"Haygarth Medical Centre, Hay Road, Talgarth Br..."
2041,202003,W96017,NaN,Haygarth Medical Centre,Hay Road,Talgarth Brecon,Powys,LD3OAW,"Haygarth Medical Centre, Hay Road, Talgarth Br..."
2819,202004,W96017,NaN,Haygarth Medical Centre,Hay Road,Talgarth Brecon,Powys,LD3OAW,"Haygarth Medical Centre, Hay Road, Talgarth Br..."
3598,202005,W96017,NaN,Haygarth Medical Centre,Hay Road,Talgarth Brecon,Powys,LD3OAW,"Haygarth Medical Centre, Hay Road, Talgarth Br..."
4376,202006,W96017,NaN,Haygarth Medical Centre,Hay Road,Talgarth Brecon,Powys,LD3OAW,"Haygarth Medical Centre, Hay Road, Talgarth Br..."
5154,202007,W96017,NaN,Haygarth Medical Centre,Hay Road,Talgarth Brecon,Powys,LD3OAW,"Haygarth Medical Centre, Hay Road, Talgarth Br..."
5931,202008,W96017,NaN,Haygarth Medical Centre,Hay Road,Talgarth Brecon,Powys,LD3OAW,"Haygarth Medical Centre, Hay Road, Talgarth Br..."
6709,202009,W96017,NaN,Haygarth Medical Centre,Hay Road,Talgarth Brecon,Powys,LD3OAW,"Haygarth Medical Centre, Hay Road, Talgarth Br..."
7490,202010,W96017,NaN,Haygarth Medical Centre,Hay Road,Talgarth Brecon,Powys,LD3OAW,"Haygarth Medical Centre, Hay Road, Talgarth Br..."


In [27]:
gpcountydata.iat[202,1] = 'Torfaen'
gpcountydata.iat[278,1] = 'Blaenau Gwent'
gpcountydata.iat[340,1] = 'Powys'
gpcountydata.iat[390,1] = 'Denbighshire'
gpcountydata.iat[391,1] = 'Denbighshire'
gpcountydata.iat[397,1] = 'Ceredigion'
gpcountydata.iat[477,1] = 'Swansea'

In [28]:
# Defunct postcodes
locountydata[locountydata['LO_County'].isnull()]

,LO_Postcode,LO_County
9,CF144TT,None
11,SA487HA,None


In [29]:
locountydata.iat[9,1] = 'Cardiff'
locountydata.iat[11,1] = 'Ceredigion'

In [30]:
# Defunct postcodes
hbcountydata[hbcountydata['HB_County'].isnull()]

,HB_Postcode,HB_County
3,CF144TT,None
4,SA313YH,None


In [31]:
hbcountydata.iat[3,1] = 'Cardiff'
hbcountydata.iat[4,1] = 'Carmarthenshire'

In [32]:
gpcountydata['GP_County'].unique()

array(['Monmouthshire', 'Rhondda Cynon Taf', 'Wrexham', 'Conwy',
       'Bridgend', 'Blaenau Gwent', 'Caerphilly', 'Newport', 'Torfaen',
       'Cardiff', 'Vale of Glamorgan', 'Pembrokeshire', 'Ceredigion',
       'Carmarthenshire', 'Gwynedd', 'Isle of Anglesey', 'Denbighshire',
       'Flintshire', 'Neath Port Talbot', 'Swansea', 'Powys',
       'Merthyr Tydfil', 'Shropshire', 'Forest of Dean District', None],
      dtype=object)

In [33]:
# Unrecognized UK County (Value captured was district council)
gpcountydata[gpcountydata['GP_County'] == 'Forest of Dean District']

,GP_Postcode,GP_County
370,GL156TN,Forest of Dean District


In [34]:
gpcountydata.iat[370,1] = 'Gloucestershire'

In [35]:
locountydata['LO_County'].unique()

array(['Monmouthshire', 'Rhondda Cynon Taf', 'Wrexham', 'Conwy',
       'Bridgend', 'Blaenau Gwent', 'Caerphilly', 'Newport', 'Torfaen',
       'Cardiff', 'Pembrokeshire', 'Ceredigion', 'Carmarthenshire',
       'Gwynedd', 'Flintshire', 'Neath Port Talbot', 'Swansea', 'Powys'],
      dtype=object)

In [36]:
hbcountydata['HB_County'].unique()

array(['Torfaen', 'Rhondda Cynon Taf', 'Flintshire', 'Cardiff',
       'Carmarthenshire', 'Neath Port Talbot', 'Powys'], dtype=object)

In [37]:
combined_data = combined_data.merge(gpcountydata,on='GP_Postcode',how='left')
                .merge(locountydata,on='LO_Postcode',how='inner')
                .merge(hbcountydata,on='HB_Postcode',how='inner')

In [38]:
# Checking for data related to unknown postcode
combined_data[combined_data['GP_Postcode'] == 'OO000OO']

,ChapterDesc,BNFChemical,HB,Locality,PracticeID,BNFCode,BNFName,Items,NIC,ActCost,...,GP_Address,LO_PracticeID,LO_Postcode,LO_Address,HB_PracticeID,HB_Postcode,HB_Address,GP_County,LO_County,HB_County
5431905,Central Nervous System,0410000D0,7A2,6A4,W92443,0410000D0AAADAD,Buprenorphine_Tab Subling 2mg S/F,1,9.22,8.5419,...,"Dermatology Clinic, Brynmeddyg Surgery, Llanyb...",6A4,SA487HA,"The Bryn, North Road, Lampeter, Ceredigion, SA...",7A2,SA313YH,"St. Davids Hospital, Jobswell Road, Carmarthen...",None,Ceredigion,Carmarthenshire


In [39]:
combined_addr[combined_addr['PracticeId'] == 'W92443']

,Period,PracticeId,Locality,Street,Area,Posttown,County,Postcode,Full address
418,202001,W92443,NaN,Dermatology Clinic,Brynmeddyg Surgery,Llanybydder,NaN,OO000OO,"Dermatology Clinic, Brynmeddyg Surgery, Llanyb..."
1192,202002,W92443,NaN,Dermatology Clinic,Brynmeddyg Surgery,Llanybydder,NaN,OO000OO,"Dermatology Clinic, Brynmeddyg Surgery, Llanyb..."
1966,202003,W92443,NaN,Dermatology Clinic,Brynmeddyg Surgery,Llanybydder,NaN,OO000OO,"Dermatology Clinic, Brynmeddyg Surgery, Llanyb..."
2744,202004,W92443,NaN,Dermatology Clinic,Brynmeddyg Surgery,Llanybydder,NaN,OO000OO,"Dermatology Clinic, Brynmeddyg Surgery, Llanyb..."
3523,202005,W92443,NaN,Dermatology Clinic,Brynmeddyg Surgery,Llanybydder,NaN,OO000OO,"Dermatology Clinic, Brynmeddyg Surgery, Llanyb..."
4301,202006,W92443,NaN,Dermatology Clinic,Brynmeddyg Surgery,Llanybydder,NaN,OO000OO,"Dermatology Clinic, Brynmeddyg Surgery, Llanyb..."
5080,202007,W92443,NaN,Dermatology Clinic,Brynmeddyg Surgery,Llanybydder,NaN,OO000OO,"Dermatology Clinic, Brynmeddyg Surgery, Llanyb..."
5858,202008,W92443,NaN,Dermatology Clinic,Brynmeddyg Surgery,Llanybydder,NaN,OO000OO,"Dermatology Clinic, Brynmeddyg Surgery, Llanyb..."
6636,202009,W92443,NaN,Dermatology Clinic,Brynmeddyg Surgery,Llanybydder,NaN,OO000OO,"Dermatology Clinic, Brynmeddyg Surgery, Llanyb..."
7417,202010,W92443,NaN,Dermatology Clinic,Brynmeddyg Surgery,Llanybydder,NaN,OO000OO,"Dermatology Clinic, Brynmeddyg Surgery, Llanyb..."


For practiceid W92443, Llanybydder posttown is located in Carmarthenshire.

In [40]:
combined_data.at[5431905,'GP_County'] = 'Carmarthenshire'
combined_data[combined_data['GP_Postcode'] == 'OO000OO']

,ChapterDesc,BNFChemical,HB,Locality,PracticeID,BNFCode,BNFName,Items,NIC,ActCost,...,GP_Address,LO_PracticeID,LO_Postcode,LO_Address,HB_PracticeID,HB_Postcode,HB_Address,GP_County,LO_County,HB_County
5431905,Central Nervous System,0410000D0,7A2,6A4,W92443,0410000D0AAADAD,Buprenorphine_Tab Subling 2mg S/F,1,9.22,8.5419,...,"Dermatology Clinic, Brynmeddyg Surgery, Llanyb...",6A4,SA487HA,"The Bryn, North Road, Lampeter, Ceredigion, SA...",7A2,SA313YH,"St. Davids Hospital, Jobswell Road, Carmarthen...",Carmarthenshire,Ceredigion,Carmarthenshire


In [41]:
# Removing unknown postcode from GP_Address column
combined_data['GP_Address'] = combined_data['GP_Address'].apply(lambda x: x.replace(', OO00 0OO','') if type(x)!=float else x)

In [42]:
# Creating additional dataframe for NADP (National Average Discounted Percentage) data
NADP_2020 = pd.DataFrame([pd.Series(combined_data['Period'].unique(),dtype='int64'),
              pd.Series([7.02,6.92,7.11,7.04,7.06,7.10,7.11,7.02,7.14,7.17,6.98,7.00])]).T \
            .rename({0:'Period',1:'NADP'},axis=1)

In [43]:
combined_data = combined_data.merge(NADP_2020, on='Period',how='inner')
combined_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7161786 entries, 0 to 7161785
Data columns (total 28 columns):
 #   Column         Dtype  
---  ------         -----  
 0   ChapterDesc    object 
 1   BNFChemical    object 
 2   HB             object 
 3   Locality       object 
 4   PracticeID     object 
 5   BNFCode        object 
 6   BNFName        object 
 7   Items          int64  
 8   NIC            float64
 9   ActCost        float64
 10  Quantity       int64  
 11  DDD            float64
 12  ADQ            float64
 13  Period         int64  
 14  BNFCode_9      object 
 15  GP_PracticeID  object 
 16  GP_Postcode    object 
 17  GP_Address     object 
 18  LO_PracticeID  object 
 19  LO_Postcode    object 
 20  LO_Address     object 
 21  HB_PracticeID  object 
 22  HB_Postcode    object 
 23  HB_Address     object 
 24  GP_County      object 
 25  LO_County      object 
 26  HB_County      object 
 27  NADP           float64
dtypes: float64(5), int64(3), object(20)
memory

In [44]:
# Checking for number of null values on every column
combined_data.isnull().sum()

ChapterDesc       0
BNFChemical       0
HB                0
Locality          0
PracticeID        0
BNFCode           0
BNFName           0
Items             0
NIC               0
ActCost           0
Quantity          0
DDD               0
ADQ               0
Period            0
BNFCode_9         0
GP_PracticeID    10
GP_Postcode      10
GP_Address       10
LO_PracticeID     0
LO_Postcode       0
LO_Address        0
HB_PracticeID     0
HB_Postcode       0
HB_Address        0
GP_County        10
LO_County         0
HB_County         0
NADP              0
dtype: int64

In [45]:
final_data = combined_data.drop(['HB','Locality','PracticeID','BNFChemical','BNFCode','BNFName','BNFCode_9','DDD','ADQ','Quantity','GP_Postcode','LO_Postcode','HB_Postcode'],axis=1)
final_data.to_csv('GP_CombinedData.csv',index=False)